In [ ]:
import pandas as pd
import operator
import time
import numpy as np

In [ ]:
config = {}

config["project_path"] = ".."
config["data_folder_path"] = "data"
config["temp_folder_path"] = "temp"

# input files
config['trade_support_input'] = '202001-04_Trade_Support.xlsx'
config['MDM_mapping'] = 'MDM_mapping.xlsx'
config['brand_to_province_brand_file'] = '202001-05_brand_to_province_brand.xlsx'

# output files
config['Trade support output'] = 'Trade support split 202001-04.xlsx'

In [ ]:
input_file_path = config["project_path"] + "/" \
                        + config["data_folder_path"] + "/" \
                        + config['trade_support_input']

ts_df = pd.read_excel(input_file_path, dtype=str)

ts_df.columns = ['YearMonth', 'SKU', 'Cost']
ts_df["Cost"] = ts_df["Cost"].astype("float")

In [ ]:
def get_brand_name(brand):
    if brand == 'NC':
        return 'Nutrilon'
    elif (brand=='AC') | (brand == 'AP'):
        return 'Aptamil'
    return ''

ts_df['Brand_Name'] = ts_df['SKU'].apply(
        lambda x: get_brand_name(x))

In [ ]:
ts_df = ts_df.groupby(by = ['YearMonth', 'Brand_Name'])['Cost'].sum().reset_index()

In [ ]:
input_file_path = config["project_path"] + "/" \
                        + config["temp_folder_path"] + "/" \
                        + config['brand_to_province_brand_file']

brand_prov_split = pd.read_excel(input_file_path, dtype=str)

brand_prov_split['Split_Ratio'] = brand_prov_split['Split_Ratio'].astype("float")

In [ ]:
ts_prov_df = pd.merge(ts_df, brand_prov_split, on=['YearMonth', 'Brand_Name'], how='left')

### Check for um-mapped records

In [ ]:
ts_prov_df[ts_prov_df['Split_Ratio'] == np.NAN]

In [ ]:
ts_prov_df['Value'] = ts_prov_df['Cost'] * ts_prov_df['Split_Ratio']

ts_prov_df = ts_prov_df[['YearMonth', 'MDM_province_code', 'Brand_Name', 'Value']]

ts_all_df = ts_prov_df.groupby(by = ['YearMonth', 'Brand_Name'])['Value'].sum().reset_index()
ts_all_df['MDM_province_code'] = 'National'

ts_prov_df = ts_prov_df.append(ts_all_df, ignore_index=True)

#### Create all year, province, KPI combination for 2020

In [ ]:
all_dates_map = pd.DataFrame(columns=['YearMonth','MDM_province_code', 'Brand_Name'])
all_types= ts_prov_df[['MDM_province_code', 'Brand_Name']].drop_duplicates()
for i in range(1,13):
    year_month = "2020" + str(i).zfill(2)
    all_types['YearMonth'] = year_month
    all_dates_map = all_dates_map.append(all_types, ignore_index=True)

In [ ]:
output_df = pd.merge(all_dates_map, ts_prov_df, on=['YearMonth','MDM_province_code', 'Brand_Name'], how='left')
output_df = output_df.fillna(0)

# Write output

In [ ]:
def format_yearmonth(year_month):
    return time.strftime("%Y/%m/%d", time.strptime(year_month,"%Y%m"))

In [ ]:
output_df["Date"] = output_df['YearMonth'].apply(lambda x : format_yearmonth(x))

output_df['Level 1'] = 'Trade Support'
output_df['Level 2'] = 'Trade Support'
output_df['Level 3'] = 'Trade Support'
output_df['Level 4'] = 'Trade Support'
output_df['Level 5'] = 'Trade Support'

output_df = output_df[['Date', 'Brand_Name', 'MDM_province_code', 'Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5', 'Value']]

output_df.columns = ['Date', 'Brand', 'Province', 'Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5', 'Value']

In [ ]:
output_file_path = config["project_path"] + "/" \
                        + config["temp_folder_path"] + "/" \
                        + config['Trade support output']

output_df.to_excel(output_file_path, index=False)